In [1]:
from riptide import *

In [2]:
#iCdJ794 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/iCdJ794.sbml')
iCdJ794 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/iCdJ794.json')
#iCdJ794 = cobra.io.read_sbml_model('/home/mjenior/Desktop/iCdJ794.sbml')

### Base Model Statistics

In [3]:
iCdJ794

Name,iCdJ794
Memory address,0x07f042d308d68
Number of metabolites,1132
Number of reactions,1129
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular"


In [4]:
iCdJ794.slim_optimize()

65.43231982360457

In [5]:
# Report some additional stats
exch = 0
for rxn in iCdJ794.reactions:
    if len(list(rxn.products)) == 0:
        exch += 1
trans = 0
for rxn in iCdJ794.reactions:
    comps = set([x.compartment for x in list(rxn.reactants)] + [x.compartment for x in list(rxn.products)])
    if len(comps) > 1:
        trans += 1
metab = len(list(iCdJ794.reactions)) - exch - trans

print('Genes: ' + str(len(list(iCdJ794.genes))))
print('Exchanges: ' + str(exch))
print('Transporters: ' + str(trans))
print('Metabolic reactions: ' + str(metab))

Genes: 794
Exchanges: 99
Transporters: 104
Metabolic reactions: 926


### RIPTiDe *in vivo* Contextualization

In [3]:
# Read in formatted transcription files
def read_transcription(infile):
    
    abund_dict = {}
    with open(infile, 'r') as abunds:
        header = abunds.readline()
        for line in abunds:
            line = line.split()
            gene = line[0].split('|')[0]
            abund = float(line[2])
            
            abund_dict[gene] = abund
            
    return abund_dict


In [4]:
# Read in in vivo C. difficile transcription
cefoperazone = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/cefoperazone_630.mapped.norm.tsv')
clindamycin = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/clindamycin_630.mapped.norm.tsv')
streptomycin = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/streptomycin_630.mapped.norm.tsv')

In [5]:
cef_iCdJ794_riptide = riptide.contextualize(model=iCdJ794, transcriptome=cefoperazone)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 276 from 1129 (75.55% change)
Metabolites pruned to 278 from 1132 (75.44% change)
Flux through the objective DECREASED to ~54.29 from ~65.43 (17.03% change)
Contextualized GENRE is concordant with the transcriptome (p=0.003 **)

RIPTiDe completed in 33 seconds



In [6]:
clinda_iCdJ794_riptide = riptide.contextualize(model=iCdJ794, transcriptome=clindamycin)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 278 from 1129 (75.38% change)
Metabolites pruned to 278 from 1132 (75.44% change)
Flux through the objective DECREASED to ~57.48 from ~65.43 (12.15% change)
Contextualized GENRE is concordant with the transcriptome (p<0.001 ***)

RIPTiDe completed in 31 seconds



In [7]:
strep_iCdJ794_riptide = riptide.contextualize(model=iCdJ794, transcriptome=streptomycin)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 280 from 1129 (75.2% change)
Metabolites pruned to 281 from 1132 (75.18% change)
Flux through the objective DECREASED to ~54.02 from ~65.43 (17.44% change)
Contextualized GENRE is concordant with the transcriptome (p<0.001 ***)

RIPTiDe completed in 1 minutes and 30 seconds



In [29]:
cef_iCdJ794_riptide.flux_samples.to_csv ('/home/mjenior/Desktop/cef_flux_samples.tsv', sep='\t', index=None, header=True)
clinda_iCdJ794_riptide.flux_samples.to_csv ('/home/mjenior/Desktop/clinda_flux_samples.tsv', sep='\t', index=None, header=True)
strep_iCdJ794_riptide.flux_samples.to_csv ('/home/mjenior/Desktop/strep_flux_samples.tsv', sep='\t', index=None, header=True)

In [30]:
cobra.io.save_json_model(cef_iCdJ794_riptide.model, "/home/mjenior/Desktop/cef_iCdJ794_riptide.json")
cobra.io.save_json_model(clinda_iCdJ794_riptide.model, "/home/mjenior/Desktop/clinda_iCdJ794_riptide.json")
cobra.io.save_json_model(strep_iCdJ794_riptide.model, "/home/mjenior/Desktop/strep_iCdJ794_riptide.json")

In [8]:
cef_iCdJ794_riptide.concordance

{'rho': 0.17768469835313902, 'p': 0.0030551983365490767, 'score': 17.8}

In [9]:
clinda_iCdJ794_riptide.concordance

{'rho': 0.23362016482762377, 'p': 8.416295906310157e-05, 'score': 23.4}

In [10]:
strep_iCdJ794_riptide.concordance

{'rho': 0.2778641948732724, 'p': 2.334522480150578e-06, 'score': 27.8}

### Context-specific Gene Essentiality

In [11]:
import copy
from cobra.flux_analysis.variability import find_essential_genes

def essential_genes(genre, media=None, optimum_fraction=0.01):
    model = copy.deepcopy(genre)
    
    # Change media conditions if necessary
    if media is not None:
        exchanges = ['EX_' + x for x in media]
        for rxn in model.reactions:
            if len(list(rxn.products)) == 0:
                if rxn.id in exchanges:
                    model.reactions.get_by_id(rxn.id).lower_bound = -1000.0
                else:
                    model.reactions.get_by_id(rxn.id).lower_bound = 0.0
    
    # Find essential genes
    essential_genes = find_essential_genes(model, threshold=optimum_fraction)
    
    return essential_genes
                

In [12]:
# Unconstrained base model
base_essential = essential_genes(iCdJ794)
base_essential_genes = set([x.id for x in base_essential])
print('Uncontextualized essential genes: ' + str(len(base_essential_genes)))

Uncontextualized essential genes: 42


In [13]:
cef_essential_genes = essential_genes(cef_iCdJ794_riptide.model)
cef_essential_genes = set([x.id for x in cef_essential_genes])
print('Cef essential genes: ' + str(len(cef_essential_genes)))

Cef essential genes: 66


In [14]:
clinda_essential_genes = essential_genes(clinda_iCdJ794_riptide.model)
clinda_essential_genes = set([x.id for x in clinda_essential_genes])
print('Clinda essential genes: ' + str(len(clinda_essential_genes)))

Clinda essential genes: 75


In [15]:
strep_essential_genes = essential_genes(strep_iCdJ794_riptide.model)
strep_essential_genes = set([x.id for x in strep_essential_genes])
print('Strep essential genes: ' + str(len(strep_essential_genes)))

Strep essential genes: 66


In [16]:
shared_essential_genes = strep_essential_genes.intersection(cef_essential_genes.intersection(clinda_essential_genes))
cef_only_essential_genes = cef_essential_genes.difference(strep_essential_genes.union(clinda_essential_genes))
strep_only_essential_genes = strep_essential_genes.difference(cef_essential_genes.union(clinda_essential_genes))
clinda_only_essential_genes = clinda_essential_genes.difference(strep_essential_genes.union(cef_essential_genes))

print('Shared: ' + str(len(shared_essential_genes)) + '\n')
print('Cef only: ' + str(len(cef_only_essential_genes)))
print('Strep only: ' + str(len(strep_only_essential_genes)))
print('Clinda only: ' + str(len(clinda_only_essential_genes)))

Shared: 63

Cef only: 3
Strep only: 1
Clinda only: 10


### Context-specific Substrate Importance

In [17]:
import pandas
from cobra.medium import minimal_medium

def find_essential_metabolites(model, fraction=0.01):

    max_growth = model.slim_optimize() * fraction
    ser = minimal_medium(model, max_growth)

    rxns = list(ser.index)
    fluxes = list(ser.values)
    cpds = []
    for x in rxns: cpds.append(model.reactions.get_by_id(x).reactants[0].name)        
    
    media = {'Reaction': rxns, 'Substrate': cpds, 'Units': fluxes}    
    media = pandas.DataFrame(media)
    print('Minimal media components: ' + str(len(media.index)))
    
    return(media)


In [18]:
cef_min_media = find_essential_metabolites(cef_iCdJ794_riptide.model)

Minimal media components: 41


In [19]:
clinda_min_media = find_essential_metabolites(clinda_iCdJ794_riptide.model)

Minimal media components: 41


In [20]:
strep_min_media = find_essential_metabolites(strep_iCdJ794_riptide.model)

Minimal media components: 42


In [21]:
cef_metabolites = set(cef_min_media['Substrate'])
strep_metabolites = set(strep_min_media['Substrate'])
clinda_metabolites = set(clinda_min_media['Substrate'])

shared_metabolites = strep_metabolites.intersection(cef_metabolites.intersection(clinda_metabolites))
cef_only_metabolites = cef_metabolites.difference(strep_metabolites.union(clinda_metabolites))
strep_only_metabolites = strep_metabolites.difference(cef_metabolites.union(clinda_metabolites))
clinda_only_metabolites = clinda_metabolites.difference(strep_metabolites.union(cef_metabolites))

print('Shared: ' + str(len(shared_metabolites)) + '\n')
print('Cef only: ' + str(len(cef_only_metabolites)))
print('Strep only: ' + str(len(strep_only_metabolites)))
print('Clinda only: ' + str(len(clinda_only_metabolites)))

Shared: 39

Cef only: 1
Strep only: 0
Clinda only: 0


###  Context-specific Topological Differences

In [22]:
# Shared topology - reactions
cef_rxns = set([x.id for x in cef_iCdJ794_riptide.model.reactions])
clinda_rxns = set([x.id for x in clinda_iCdJ794_riptide.model.reactions])
strep_rxns = set([x.id for x in strep_iCdJ794_riptide.model.reactions])

shared_rxns = strep_rxns.intersection(cef_rxns.intersection(clinda_rxns))
cef_only_rxns = cef_rxns.difference(strep_rxns.union(clinda_rxns))
strep_only_rxns = strep_rxns.difference(cef_rxns.union(clinda_rxns))
clinda_only_rxns = clinda_rxns.difference(strep_rxns.union(cef_rxns))

print('Shared: ' + str(len(shared_rxns)) + '\n')
print('Cef only: ' + str(len(cef_only_rxns)))
print('Strep only: ' + str(len(strep_only_rxns)))
print('Clinda only: ' + str(len(clinda_only_rxns)))

Shared: 261

Cef only: 3
Strep only: 1
Clinda only: 5


In [23]:
# Shared topology - metabolites
cef_cpds = set([x.id for x in cef_iCdJ794_riptide.model.metabolites])
clinda_cpds = set([x.id for x in clinda_iCdJ794_riptide.model.metabolites])
strep_cpds = set([x.id for x in strep_iCdJ794_riptide.model.metabolites])

shared_cpds = strep_cpds.intersection(cef_cpds.intersection(clinda_cpds))
cef_only_cpds = cef_cpds.difference(strep_cpds.union(clinda_cpds))
strep_only_cpds = strep_cpds.difference(cef_cpds.union(clinda_cpds))
clinda_only_cpds = clinda_cpds.difference(strep_cpds.union(cef_cpds))

print('Shared: ' + str(len(shared_cpds)) + '\n')
print('Cef only: ' + str(len(cef_only_cpds)))
print('Strep only: ' + str(len(strep_only_cpds)))
print('Clinda only: ' + str(len(clinda_only_cpds)))

Shared: 270

Cef only: 1
Strep only: 0
Clinda only: 4


### Reaction essentiality

In [24]:
import copy
from cobra.flux_analysis.variability import find_essential_reactions

def essential_reactions(genre, media=None, optimum_fraction=0.01):
    model = copy.deepcopy(genre)
    
    # Change media conditions if necessary
    if media is not None:
        exchanges = ['EX_' + x for x in media]
        for rxn in model.reactions:
            if len(list(rxn.products)) == 0:
                if rxn.id in exchanges:
                    model.reactions.get_by_id(rxn.id).lower_bound = -1000.0
                else:
                    model.reactions.get_by_id(rxn.id).lower_bound = 0.0
    
    # Find essential genes
    essential_reactions = find_essential_reactions(model, threshold=optimum_fraction)
    
    return essential_reactions
                

In [25]:
cef_essential_reactions = essential_reactions(cef_iCdJ794_riptide.model)
cef_essential_reactions = set([x.id for x in cef_essential_reactions])
print('Cef essential reactions: ' + str(len(cef_essential_reactions)))

Cef essential reactions: 210


In [26]:
clinda_essential_reactions = essential_reactions(clinda_iCdJ794_riptide.model)
clinda_essential_reactions = set([x.id for x in clinda_essential_reactions])
print('Clinda essential reactions: ' + str(len(clinda_essential_reactions)))

Clinda essential reactions: 233


In [27]:
strep_essential_reactions = essential_reactions(strep_iCdJ794_riptide.model)
strep_essential_reactions = set([x.id for x in strep_essential_reactions])
print('Strep essential reactions: ' + str(len(strep_essential_reactions)))

Strep essential reactions: 212


In [28]:
cef_only_essential_reactions = cef_essential_reactions.difference(strep_essential_reactions.union(clinda_essential_reactions))
strep_only_essential_reactions = strep_essential_reactions.difference(cef_essential_reactions.union(clinda_essential_reactions))
clinda_only_essential_reactions = clinda_essential_reactions.difference(strep_essential_reactions.union(cef_essential_reactions))
print('Cef only: ' + str(len(cef_only_essential_reactions)))
print('Strep only: ' + str(len(strep_only_essential_reactions)))
print('Clinda only: ' + str(len(clinda_only_essential_reactions)))

cef_strep_reactions = strep_essential_reactions.intersection(cef_essential_reactions)
cef_strep_reactions = cef_strep_reactions.difference(clinda_essential_reactions)
cef_clinda_reactions = clinda_essential_reactions.intersection(cef_essential_reactions)
cef_clinda_reactions = cef_strep_reactions.difference(strep_essential_reactions)
clinda_strep_reactions = clinda_essential_reactions.intersection(strep_essential_reactions)
clinda_strep_reactions = clinda_strep_reactions.difference(cef_essential_reactions)
print('\nCef + Strep: ' + str(len(cef_strep_reactions)))
print('Cef + Clinda: ' + str(len(cef_clinda_reactions)))
print('Clinda + Strep: ' + str(len(clinda_strep_reactions)))

shared_essential_reactions = strep_essential_reactions.intersection(cef_essential_reactions.intersection(clinda_essential_reactions))
print('\nCef + Strep + Clinda: ' + str(len(shared_essential_reactions)))

Cef only: 11
Strep only: 7
Clinda only: 30

Cef + Strep: 2
Cef + Clinda: 0
Clinda + Strep: 6

Cef + Strep + Clinda: 197


### Analysis of Constrained Flux Samples

In [ ]:
# Results from flux distribution analysis of shared reactions
unchanged = 